In [1]:
import tensorflow as tf
import utility.initialize_tf
from custom_models.gamornet_keras_module import *
from utility.data_loading import *
import numpy as np
from sklearn.model_selection import train_test_split
from utility.display_prediction_results import *

2.5.0
2.5.0


In [2]:
training_data = "fd=0.3-0.7_hdf5"
transfer_data = "califa_s_1.0"
testing_data = "nair_abraham_2010_hdf5"

In [3]:
x_dataset, y_dataset, metadata = utility.data_loading.load_hdf5_data(name=training_data)
x_dataset = np.expand_dims(x_dataset, axis=1)
y_dataset = np.expand_dims(y_dataset, axis=1)
input_shape = x_dataset.shape[1:]

E:\Anaconda3\envs\hon-tf-2.5.0\lib\site-packages\tables\path.py:155: NaturalNameWarning: object name is a Python keyword: 'class'; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


(54000, 100, 100)


In [4]:
base_model = gamornet_build_model_keras(input_shape=input_shape)

In [5]:
base_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 1, 25, 96)         1161696   
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 13, 96)         0         
_________________________________________________________________
local_response_normalization (None, 1, 13, 96)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 13, 256)        614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 7, 256)         0         
_________________________________________________________________
local_response_normalization (None, 1, 7, 256)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 7, 384)         8

In [6]:
x_train, x_val, y_train, y_val = train_test_split(x_dataset, y_dataset, shuffle=True, train_size=0.8)
trained_model = gamornet_train_keras(x_train, y_train, x_val, y_val, input_shape=input_shape)

Epoch 1/100
675/675 [==============================] - 76s 49ms/step - loss: 0.7355 - accuracy: 0.4961 - val_loss: 0.7010 - val_accuracy: 0.5051
Epoch 2/100
675/675 [==============================] - 26s 38ms/step - loss: 0.7101 - accuracy: 0.5007 - val_loss: 0.6985 - val_accuracy: 0.4949
Epoch 3/100
675/675 [==============================] - 25s 37ms/step - loss: 0.7055 - accuracy: 0.5024 - val_loss: 0.6972 - val_accuracy: 0.5051
Epoch 4/100
675/675 [==============================] - 25s 37ms/step - loss: 0.7025 - accuracy: 0.5081 - val_loss: 0.6965 - val_accuracy: 0.4949
Epoch 5/100
675/675 [==============================] - 26s 39ms/step - loss: 0.7014 - accuracy: 0.5056 - val_loss: 0.6949 - val_accuracy: 0.4949
Epoch 6/100
675/675 [==============================] - 25s 37ms/step - loss: 0.6996 - accuracy: 0.5080 - val_loss: 0.6937 - val_accuracy: 0.5138
Epoch 7/100
675/675 [==============================] - 25s 37ms/step - loss: 0.6975 - accuracy: 0.5165 - val_loss: 0.6919 - val_ac

In [ ]:
x_transfer, y_transfer, transfer_metadata = load_hdf5_data(transfer_data)
x_transfer = np.expand_dims(x_transfer, axis=1)
y_transfer = np.expand_dims(y_transfer, axis=1)
input_shape = x_transfer.shape[1:]
transfer_datagen = create_datagen(x_transfer)
transferred_model, transfer_history = gamornet_tl_keras(transfer_datagen, input_shape = input_shape, load_layers_bools="load_bools_SDSS", trainable_bools="train_bools_SDSS", 
    files_save_path="./saved_models/gamornet/", batch_size=32, loss='sparse_categorical_crossentropy')
plot_evaluation_results(transfer_history)
transferred_model.save("./saved_models/gamornet/transferred_model.hdf5")

In [ ]:
tuned_model, tuning_history = gamornet_tl_keras(transfer_datagen, input_shape = input_shape, load_layers_bools="load_bools_SDSS", trainable_bools="train_bools_SDSS", 
    files_save_path="./saved_models/gamornet/", batch_size=32, loss='sparse_categorical_crossentropy', epochs=10)
plot_evaluation_results(tuning_history)
tuned_model.save("./saved_models/gamornet/tuned_model.hdf5")